In [2]:
import random
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from utils_text import preprocess_text, preprocess_text_stop

## Importando Dataset

In [5]:
df = pd.read_csv('scrapper_results.csv')

In [6]:
df.head()

,name,raw_review,date
0,Otilio Davini,Liberação para efetuar pagamentos pelo app é r...,27 de janeiro de 2020
1,Irapuan Braga,"O sistema BB Code, quando funciona, é bom. Mas...",20 de janeiro de 2020
2,Allan,"App é bem interativo, te dá bastante opções de...",10 de janeiro de 2020
3,Diego Santos,Excelente! Tenho algumas sugestões de melhoria...,26 de janeiro de 2020
4,Mateus Jiandotti Gonçalves,"Bom dia, já tem muito tempo que uso o app do B...",23 de janeiro de 2020


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2760 entries, 0 to 2759
Data columns (total 3 columns):
name          2760 non-null object
raw_review    2760 non-null object
date          2760 non-null object
dtypes: object(3)
memory usage: 64.8+ KB


In [9]:
df['clean_review'] = df['raw_review'].apply(preprocess_text_stop)

In [10]:
df.head()

,name,raw_review,date,clean_review
0,Otilio Davini,Liberação para efetuar pagamentos pelo app é r...,27 de janeiro de 2020,liberacao efetuar pagamentos app ruim vc nao c...
1,Irapuan Braga,"O sistema BB Code, quando funciona, é bom. Mas...",20 de janeiro de 2020,sistema bb code funciona bom algum problema ce...
2,Allan,"App é bem interativo, te dá bastante opções de...",10 de janeiro de 2020,app bem interativo bastante opcoes transacoes ...
3,Diego Santos,Excelente! Tenho algumas sugestões de melhoria...,26 de janeiro de 2020,excelente algumas sugestoes melhoria notificac...
4,Mateus Jiandotti Gonçalves,"Bom dia, já tem muito tempo que uso o app do B...",23 de janeiro de 2020,bom dia ja tempo uso app banco brasil porem co...


## Count Vectorizer

In [11]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=['bb'])

In [13]:
dtm = cv.fit_transform(df['clean_review'])

In [16]:
dtm

<2760x2221 sparse matrix of type '<class 'numpy.int64'>'
	with 33477 stored elements in Compressed Sparse Row format>

In [14]:
len_words = len(cv.get_feature_names())

In [15]:
for i in range(10):
    random_word_id = random.randint(0,len_words)
    print(cv.get_feature_names()[random_word_id])

novas
existem
finais
mandaram
chave
alteracoes
aconteceu
sido
populacao
fechando


## LDA

In [28]:
LDA = LatentDirichletAllocation(n_components=10,random_state=42)

In [29]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [30]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['entrar', 'nada', 'sendo', 'corrente', 'so', 'consigo', 'dinheiro', 'fazer', 'banco', 'ja', 'pra', 'aplicativo', 'app', 'conta', 'nao']


THE TOP 15 WORDS FOR TOPIC #1
['excelente', 'credito', 'aplicativo', 'cartao', 'gastos', 'tempo', 'nao', 'app', 'preciso', 'vida', 'transferencias', 'fazer', 'facilita', 'pagamentos', 'bom']


THE TOP 15 WORDS FOR TOPIC #2
['bom', 'sempre', 'ainda', 'pagamento', 'outros', 'falta', 'nenhum', 'ate', 'informacoes', 'so', 'credito', 'aplicativo', 'cartao', 'app', 'nao']


THE TOP 15 WORDS FOR TOPIC #3
['perfil', 'ser', 'aplicativo', 'parte', 'conta', 'sempre', 'pra', 'porem', 'poderia', 'opcao', 'fica', 'geral', 'nao', 'bom', 'app']


THE TOP 15 WORDS FOR TOPIC #4
['agencia', 'casa', 'excelente', 'resolver', 'melhor', 'dia', 'brasil', 'ir', 'bom', 'otimo', 'parabens', 'app', 'tudo', 'aplicativo', 'banco']


THE TOP 15 WORDS FOR TOPIC #5
['eficiente', 'otimo', 'acesso', 'intuitivo', 'bem', 'excelente', 'seguro', 'simples', '

## Seeing results on DataFrame

In [31]:
topic_results = LDA.transform(dtm)

In [32]:
df['topic'] = topic_results.argmax(axis=1)

In [33]:
df.head(10)

,name,raw_review,date,clean_review,topic
0,Otilio Davini,Liberação para efetuar pagamentos pelo app é r...,27 de janeiro de 2020,liberacao efetuar pagamentos app ruim vc nao c...,6
1,Irapuan Braga,"O sistema BB Code, quando funciona, é bom. Mas...",20 de janeiro de 2020,sistema bb code funciona bom algum problema ce...,2
2,Allan,"App é bem interativo, te dá bastante opções de...",10 de janeiro de 2020,app bem interativo bastante opcoes transacoes ...,2
3,Diego Santos,Excelente! Tenho algumas sugestões de melhoria...,26 de janeiro de 2020,excelente algumas sugestoes melhoria notificac...,2
4,Mateus Jiandotti Gonçalves,"Bom dia, já tem muito tempo que uso o app do B...",23 de janeiro de 2020,bom dia ja tempo uso app banco brasil porem co...,7
5,Sergio Augusto Fanchioti Loureiro,"App travou há 02 dias, e tive que desinstalar....",24 de janeiro de 2020,app travou ha 02 dias desinstalar reinstalar t...,6
6,Vanderlei Ortencio,O inicio foi tranquilo. Facilidade para baixar...,20 de janeiro de 2020,inicio tranquilo facilidade baixar instalar in...,0
7,Lúcio Silva,Muito bom. Uma agência na palma da mão. De tud...,10 de janeiro de 2020,bom agencia palma mao tudo pouco evitando fila...,1
8,Paulo Júnior,O App do Banco do Brasil é excelente! É necess...,22 de janeiro de 2020,app banco brasil excelente necessario avaliar ...,4
9,Renato Batista,"Aplicativo esteticamente agradável, mas só est...",11 de janeiro de 2020,aplicativo esteticamente agradavel so estetica...,7


In [44]:
example = df.iloc[220]

In [45]:
example['raw_review']

'Estou muito satisfeito com esse aplicativo do banco do Brasil. Abri facilmente a conta e já consegui fazer até transferência para outros bancos.'

In [46]:
example['topic']

7

In [47]:
example_words = LDA.components_[example['topic']].argsort()[-15:]

In [48]:
print([cv.get_feature_names()[i] for i in example_words])

['fazer', 'sempre', 'banco', 'todas', 'agencia', 'ter', 'ir', 'atende', 'necessidades', 'pra', 'nao', 'app', 'conta', 'bom', 'aplicativo']
